<a href="https://colab.research.google.com/github/adilsonalbino/SQL-e-PYSPARK/blob/main/11_Realizando_Buscas_filtros_com_EXISTS%2C_e_LEFT_Semi_JOINS_com_SQL_e_PYSPARK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#11-Realizando Buscas/filtros com EXISTS, e LEFT Semi JOINS com SQL e PYSPARK


*   A ideia é filtrar dados de uma tabela somente se esses dados existir na tabela principal.



In [5]:
#instalando pyspark
!pip install pyspark

#importando bibliotecas
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

#Criando app spark
spark = SparkSession.builder \
.appName("Spark")\
.getOrCreate()

In [33]:
file_path_carros = "/content/drive/MyDrive/Colab Notebooks/PySpark_SQL/DATASET_CARROS/modelo_carro.csv"


In [41]:
df_carros = spark\
.read.format("csv")\
.option("header", True)\
.load(file_path_carros)\

df_carros.show(5)

+--------+------------+---------+---------+
|id_carro|modelo_carro|    preco|cod_marca|
+--------+------------+---------+---------+
|       1|      Avalon|$78401.95|       54|
|       2|         RDX|$95987.38|        1|
|       3|        Golf|$61274.55|       55|
|       4|          EX|$84981.12|       23|
|       5|      Escort|$77466.89|       17|
+--------+------------+---------+---------+
only showing top 5 rows



In [51]:
#Criando dois dataframes fitrados com where()

df_carros_inicio = df_carros.where(
    (col("id_carro") == '1') |
    (col("id_Carro") == '2') |
    (col("id_carro") == '3')
)

df_carros_fim = df_carros.where(
    (col("id_carro") == '1') |
    (col("id_Carro") == '2') |
    (col("id_carro") == '3') |
    (col("id_carro") == '4')

)

df_carros_inicio.show()
df_carros_fim.show()

+--------+------------+---------+---------+
|id_carro|modelo_carro|    preco|cod_marca|
+--------+------------+---------+---------+
|       1|      Avalon|$78401.95|       54|
|       2|         RDX|$95987.38|        1|
|       3|        Golf|$61274.55|       55|
+--------+------------+---------+---------+

+--------+------------+---------+---------+
|id_carro|modelo_carro|    preco|cod_marca|
+--------+------------+---------+---------+
|       1|      Avalon|$78401.95|       54|
|       2|         RDX|$95987.38|        1|
|       3|        Golf|$61274.55|       55|
|       4|          EX|$84981.12|       23|
+--------+------------+---------+---------+



#Utilizando SQL

In [54]:
#Criando tabelas teporárias com os dfs
df_carros_inicio.createOrReplaceTempView("carros_inicio")
df_carros_fim.createOrReplaceTempView("carros_fim")

In [56]:
#Utilizando EXISTS
#A função permite retornar apenas os dados existente que bater com o ID na outra tabela
df_carros_fim_sql = spark.sql("""
SELECT
  *
FROM carros_fim a
WHERE EXISTS (SELECT * FROM carros_inicio b WHERE a.id_carro = b.id_carro)
""")

df_carros_fim_sql.show()

+--------+------------+---------+---------+
|id_carro|modelo_carro|    preco|cod_marca|
+--------+------------+---------+---------+
|       1|      Avalon|$78401.95|       54|
|       2|         RDX|$95987.38|        1|
|       3|        Golf|$61274.55|       55|
+--------+------------+---------+---------+



#Utilizando PYSPARK
Obs: até o momento a função EXISTS não é aplicável diretamente para operações de DataFrames no PySpark. Em vez disso, você deve usar a operação de junção (join) passando no finalo argumento "leftsemi"

In [66]:
#Utilizando o LEFTSEMI para trazer o mesmo efeito do EXISTS

df_carros_spark = df_carros_fim.join(
    df_carros_inicio,
    df_carros_fim.id_carro == df_carros_inicio.id_carro,
    "leftsemi"
    ).show()

+--------+------------+---------+---------+
|id_carro|modelo_carro|    preco|cod_marca|
+--------+------------+---------+---------+
|       1|      Avalon|$78401.95|       54|
|       2|         RDX|$95987.38|        1|
|       3|        Golf|$61274.55|       55|
+--------+------------+---------+---------+

